**ESCP Business School, Mim Specialisation Finance de Marché (OP58) class 2022 <br> Course: Coding With Python, taught by O.Chevillon**

**Team:** David Attias (Student ID: e212386)

# Pairs Trading: Performance of a Relative Value Arbitrage Rule

### ABSTRACT:

##### In this notebook, we will try to reproduce the Pair Trading Strategy described by Evan Gatev, William N. Goetzmann and K. Geert Rouwenhorst in their paper Pairs Trading: Performance of a Relative Value Arbitrage Rule, and most specially the version published in 2006.

##### We will operate by first defining functions that will allow the user of this notebook to reproduce the Pair Trading Strategy easily on the dataset of their choice.

##### The principles of Pair Trading Strategies are easy to understand: finding two assets that move closely together over a period of time, or in Econometrics terms, are "cointegrated", and betting, when the spread between the two assets widens beyond a certain threshold, that it will converge back to its mean value.

##### The Strategy described in the article initiates trading decisions is trying to be as simplistic as possible in order to underline the large arbitraging opportunities using pair trading from the 1960s to the beginning of the 1990s when they started to decline. Because we didn't manage to find satisfactory data sources detailling the composition of the S&P 500 for past periods, we will content ourselves of using the set of companies listed in the current S&P 500.

In [4]:
!pip install yfinance
!pip install statsmodels
!pip install pandas_datareader


## Imports

In [5]:
import numpy as np
import pandas as pd
import statsmodels


import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

In [6]:
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
import datetime
import yfinance as yf
yf.pdr_override()

## Step 1: Let's find some Data

#### Our method goes as follows:
- We import the table containing the current composition of the S&P 500 from Wikipedia, containing the tickers and the GICS Sector
- We cluster our Data by GICS Sector, in order to reduce future computing effort.<br> Four sectors are proposed to the appreciation of the reader:

    - Utilities
    - Financials
    - Information Technology
    - Industrials

> The link of reference (last consulted the 6th of May 2022) : https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [7]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
reference_table = wiki[0]

Utilities=reference_table[reference_table['GICS Sector']=='Utilities']
Financials=reference_table[reference_table['GICS Sector']=='Financials']
Information_Technology=reference_table[reference_table['GICS Sector']=='Information Technology']
Industrials=reference_table[reference_table['GICS Sector']=='Industrials']

#For interest, let's see the table available on wikipedia.
reference_table

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
499,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
502,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


#### We now import the values of the of the different stocks  referenced as in the Financial Sector, for at least 378 values (which corresponds to a ), we therefore take a larger dataset that we will eventually shorten via ".iloc(:378)" function, any other variable than 378 would have also worked obviously.

#### Let's interess ourselves to the Financials sector from 2000 to half 2001.

> #### We will only use adjusted closes prices for this strategy



In [8]:
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2002, 1, 1)

tickers = Financials.Symbol.tolist()

data = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
data = data.iloc[:378]
#An error message may appear, to warn us from missing vaklues
#We remove the stocks with missing values using the .dropna() function
data.dropna(inplace=True,axis=1)
data

[*********************100%***********************]  66 of 66 completed

14 Failed downloads:
- CME: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- SBNY: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- SYF: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- AIZ: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- BRK.B: No data found, symbol may be delisted
- CBOE: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- AMP: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- MKTX: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- CFG: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- ICE: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- MSCI: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- DFS: Data doesn't exist for startDate = 946681200, endDate = 1009839600
- FRC: Data doesn't exist for startDate = 

,AFL,AIG,AJG,ALL,AON,AXP,BAC,BEN,BK,BLK,...,SIVB,SPGI,STT,TFC,TROW,TRV,USB,WFC,WRB,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2001-10-23,8.395300,1077.697021,19.163189,20.133278,29.129017,19.800785,17.916464,6.394767,23.548868,27.539675,...,20.590000,18.611792,31.558790,15.831015,8.647800,29.240181,9.329957,11.216026,3.518141,32.874516
2001-10-24,8.372315,1077.439819,19.380781,19.902700,29.055994,19.624104,18.232763,6.464315,23.911673,27.546247,...,22.330000,18.849485,31.981922,16.050556,8.488548,29.442041,9.604371,11.300151,3.493475,33.350433
2001-10-25,8.388735,1078.596680,19.309925,19.781334,29.048704,20.069075,18.488234,6.472264,24.234179,27.487085,...,23.049999,18.981915,33.265442,16.221306,8.350327,29.032389,9.963215,11.608594,3.576559,34.331116
2001-10-26,8.323068,1077.439819,19.188496,19.732798,29.917465,20.101789,18.445662,6.587523,24.173700,27.539675,...,23.860001,18.873262,33.392357,16.333511,8.386385,29.661716,9.773238,11.524468,3.609014,34.936848
2001-10-29,8.273821,1041.438965,19.051861,19.653919,29.873672,19.591385,18.095911,6.420600,23.380903,27.934038,...,23.760000,18.340128,32.221718,15.992008,8.311267,29.281750,9.393282,11.216026,3.495422,34.843094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-04-17,9.324547,1008.322388,12.878181,24.623047,24.956034,25.384886,15.524601,8.477597,33.325527,21.689953,...,23.590000,21.347235,34.754627,16.716768,10.197304,25.151754,11.602994,12.677503,2.813493,37.258221
2001-04-18,9.308185,1027.717529,12.652514,25.536352,25.280226,27.729303,16.473988,8.923782,34.177467,22.610140,...,25.070000,21.866163,36.699471,17.115589,11.177116,25.497675,11.670426,13.204577,2.784470,38.153145
2001-04-19,9.344175,1048.783569,13.189102,26.587837,25.273018,27.976776,16.838223,8.775058,34.743221,22.412956,...,26.400000,21.552782,37.784237,17.125200,11.004389,25.415581,11.514817,13.307220,2.709004,39.556427


In [9]:
data.describe()

,AFL,AIG,AJG,ALL,AON,AXP,BAC,BEN,BK,BLK,...,SIVB,SPGI,STT,TFC,TROW,TRV,USB,WFC,WRB,ZION
count,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,...,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000,378.000000
mean,8.816587,1045.879986,12.140515,18.577766,23.640343,31.727203,14.903007,7.152365,30.613820,21.903048,...,35.341410,18.997522,36.331628,14.433420,11.348032,24.147151,11.133867,12.087726,2.073125,35.606195
std,1.540384,133.273874,3.628198,4.477418,3.434415,5.426422,1.988894,0.924942,4.640719,5.762968,...,10.159483,1.695681,5.578657,2.228069,1.456389,5.101081,1.204245,1.335386,0.830199,4.357700
min,5.591139,696.183960,5.742354,10.437093,14.784334,19.257666,11.221374,4.952282,20.282543,10.352022,...,17.000000,14.162580,22.066078,10.189417,8.203093,12.325465,7.984434,8.424982,0.920350,25.613075
25%,7.739476,980.427826,9.172565,14.021433,21.684612,28.582770,13.539047,6.454439,26.766217,16.082605,...,27.231093,18.086868,33.150980,12.541110,10.242562,20.076767,10.068367,11.246854,1.333344,32.155251
50%,8.683523,1041.156982,12.668177,19.228215,24.640949,32.827057,14.683618,7.140230,31.481545,23.209892,...,32.781250,19.363527,36.925241,13.788712,11.288357,26.273350,11.338378,12.120858,2.081562,35.814365
75%,9.991666,1128.248779,14.591861,22.805687,26.039953,35.874416,15.727150,7.754151,34.492696,26.948130,...,42.000000,20.096635,39.633291,16.714287,12.503464,28.028471,12.063917,12.956685,2.718892,38.799717
max,11.921671,1331.269775,19.380781,26.587837,29.917465,40.824825,19.765591,9.469133,39.157940,31.220396,...,64.062500,22.431362,46.824036,18.140305,14.323285,32.665760,13.973625,15.393949,3.777781,44.769630


## Step 2: Forming the Pairs

#### In order to form the pairs, Gatev, Goetzmann and Rouwenhorst decided to use normalized the data from their training period. <br> We do so by first splitting our dataset between the number of days of the training phase and of the testing phase. In our case, those values will be 252 days (a year) for the first one and 126 (6 months) for the later, in accordance to the article method.

In [10]:

training_df = data.iloc[:252]
testing_df = data.iloc[252:]


def normalize_data(df):
    #normalize, per column, the values of a dataframe.
    min = df.min()
    max = df.max()
    x = df 
    
    # time series normalization part
    # y will be a column in a dataframe
    y = (x - min) / (max - min)
    
    return y

#We therefore create a normalized version of our training dataset for the pair formation period.


## Stationarity and Cointegration
#### First, we need to check for the non-stationarity of the asset prices, a prepondary contition to checking for cointegration tests.

#### The test that will be used for stationarity is the Augmented Dicky Fuller, which aims at testing whether past prices impact future prices.

In [11]:
#performs an Augmented Dickey Fuller test and removes stationary variables from the dataset.
def stationarity_test(df, cutoff=0.01):
    
    # We start by normalizing the data on which we perform the test
    normal=normalize_data(df)    
    
    # We store the variables to remove in the following list
    to_remove=[]
    
    for a in normal.columns:
        # H_0 in adfuller is unit root exists (non-stationary)
        # We must observe significant p-value to convince ourselves that the series is stationary
        pvalue = adfuller(normal[a])[1]
        if pvalue < cutoff:
            to_remove.append(a)
        df.drop([a],axis=1)
    if len(to_remove)==0:
        print('None of the studied variables was stationnary at a ',cutoff*100,'percent confidence-level, we can therefore proceed with cointegration tests')
    else: 
        print('The following variables were stationnary at a ',cutoff*100,'percent confidence-level, and were therefore removed from the dataset')
        print(to_remove)


def find_cointegrated_pairs(df):
    
    # We start by normalizing the data on which we perform the test
    normal=normalize_data(df)    
    
    n = normal.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = normal[keys[i]]
            S2 = normal[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

#### We perform the aforementioned tests on our training datasets.

In [12]:
stationarity_test(training_df)
scores, pvalue, pairs=find_cointegrated_pairs(data)
print('There are ', len(pairs),' pairs of cointegrated variables.')
print(pairs)


None of the studied variables was stationnary at a  1.0 percent confidence-level, we can therefore proceed with cointegration tests
There are  91  pairs of cointegrated variables.
[('AFL', 'CB'), ('AFL', 'IVZ'), ('AIG', 'BK'), ('AIG', 'HIG'), ('AIG', 'IVZ'), ('AIG', 'STT'), ('AJG', 'CB'), ('AJG', 'FITB'), ('AJG', 'LNC'), ('ALL', 'PNC'), ('ALL', 'WFC'), ('AON', 'BK'), ('AON', 'BLK'), ('AON', 'C'), ('AON', 'CB'), ('AON', 'CMA'), ('AON', 'COF'), ('AON', 'HIG'), ('AON', 'LNC'), ('AON', 'MMC'), ('AON', 'SPGI'), ('AON', 'TRV'), ('AXP', 'SIVB'), ('AXP', 'TROW'), ('BEN', 'PNC'), ('BEN', 'SPGI'), ('BK', 'HIG'), ('BK', 'IVZ'), ('BK', 'TRV'), ('BLK', 'CB'), ('BLK', 'HIG'), ('BLK', 'LNC'), ('BLK', 'TRV'), ('BRO', 'FITB'), ('BRO', 'MCO'), ('BRO', 'PGR'), ('C', 'COF'), ('C', 'HIG'), ('C', 'IVZ'), ('C', 'MMC'), ('C', 'TRV'), ('CB', 'CMA'), ('CB', 'COF'), ('CB', 'HIG'), ('CB', 'LNC'), ('CB', 'TRV'), ('CB', 'WFC'), ('COF', 'IVZ'), ('COF', 'SPGI'), ('FDS', 'FITB'), ('FDS', 'GL'), ('FDS', 'HBAN'), ('FDS'

#### We create functions to compute the Residuals from the regression between a pair of cointegrated actions, as well as their Sum of Squared Errors (SSE) which the researchers chose as a rule to rank the different pairs and test whether we could observe a greater performance of pairs with lower SSE values.

In [13]:
#Gives the residual from the regression from two assets X and Y
def Spread(Y,X):
  model = sm.OLS(Y, X).fit()
  spread = model.fittedvalues - Y
  return spread

#Returns the SSE of the regression of Y on X
def SSE(Y,X):
  sse= np.sum((Spread(Y,X) - Y)**2)
  return sse

#Creates a dataframe containing the Sum of Squared residuals of the different pairs, ranked from the lowest to the highest.
def SSE_Vector(pairs,df) :
  
  SqErr=[]
  for i in range(len(pairs)):
    Item = pairs[i]
    S1= Item[0]
    S2= Item[1]
    spr=SSE(df[S1],df[S2])
    SqErr.append(spr)
  df2=pd.DataFrame(list(zip(pairs,SqErr)),columns=['Pairs','SSE'])
  df3=df2.sort_values("SSE",ignore_index=True)
  return df3
  

#### The Pairs are separated in 3 groups, the first 5, the first 20 and those ranked from the 101th to the 120th positions, according to the Sum of Squared Errors (SSE) of the OLS model of their normalized data, in conformity to the paper. 

In [14]:
ranked_pairs=SSE_Vector(pairs,data)
pairs=ranked_pairs['Pairs']
group_1=pairs[:5]
group_2=pairs[:20]

#for the 3rd group we need to reset the index
group_3=pairs[100:120].reset_index()
group_3=group_3['Pairs']

ranked_pairs

,Pairs,SSE
0,"(BRO, PGR)",4.349713e+03
1,"(BRO, MCO)",4.589661e+03
2,"(BRO, FITB)",4.842344e+03
3,"(BEN, SPGI)",2.004557e+04
4,"(BEN, PNC)",2.035771e+04
...,...,...
86,"(C, IVZ)",3.498875e+07
87,"(AIG, STT)",4.258856e+08
88,"(AIG, BK)",4.278543e+08
89,"(AIG, HIG)",4.297835e+08


## Step 3: Finding Monthly and Total Profits

#### The strategy chosen by Gatev, Goetzmann and Rouwenhorst for the paper is a very simplistic form of Pair Trading.

- The data is devided in two periods, training and testing
- The training period, of 12 months, is meant to form the pairs, and provide historical mean and volatility
- The agent opens a position whenever the Normalized Spread deviates from two standard deviations from its mean value.<br> He invests in the portfolio $ P = Stock_A - \beta Stock_B $ 


#### Intuitively, the strategy is to:

- short the portfolio P when $ P > E(P_Y)+ 2 \sigma_Y $ 
- long the portfolio P when $ P < E(P_Y)- 2 \sigma_Y $
- close the position as soon as the Spread converges to the historical mean
- close the position at the end of each month if the reversal hasn't already occured.

#### where $E(P_Y)$ , $\beta$ and $ \sigma_m $ are historical, the first value being the yearly average and the second being the 1-month volatility from the training period.

#### Despite our purpose here was to stay as close as possible to the approach from the researchers, we must acknowledge that many algortithms available online consider moving averages, moving volatility and normal distribution as part of their trading decision rule. Those models appear more sophisticated and could allegdgedly provide better returns, in opposite to the approach of the article, which focuses on very rudimentary pair trading strategies.

>To learn more about it: https://github.com/KidQuant/Pairs-Trading-With-Python/blob/master/PairsTrading.ipynb

In [15]:
# provides a matrix with the monthly returns of the trading strategy aforementioned, for a list "pairs" of the pairs we want to study.

def Profit_Matrix(df, pairs,training_days=252,testing_days=126):
    
    training_df = df.iloc[:training_days]
    testing_df = df.iloc[training_days:(training_days+testing_days)]
        
    monthly_profit=0
    w=0 #weight given to the Portfolio P = y - B*X 
    
    #We want to know the profits as well as the capital employed for the strategy
    res=pd.DataFrame(columns=pairs)
    weights_mtx=pd.DataFrame(columns=pairs)
    
    for i in range(len(pairs)):
        
        pair_i=pairs[i]
        Stock_A=pair_i[0]
        Stock_B=pair_i[1]
        
        X=sm.add_constant(training_df[Stock_B])


        model = sm.OLS(training_df[Stock_A],X).fit()
        beta=model.params
        
        hist_spreads = training_df[Stock_A]-model.fittedvalues
        spread= testing_df[Stock_A]-beta[1]*testing_df[Stock_B]
        
        #One Year historical average, one month historical volatility
        average= np.mean(hist_spreads)
        vol= np.std(hist_spreads)/3.4641
        
        #We want to know the profits as well as the capital employed for the strategy
        profits=[]
        weight_series=[] 
        monthly_return_series=[]
        for a in range(testing_days):
            
            if a > 0 :
                
                if w == 0:
                    if spread[a] > average + 2*vol :  #if the error is too large, we short it, equating a negative weight in our portfolio
                        w=-1
                    elif spread[a] < average - 2*vol : #if the error is "too negative", we long it, equating a positive weight in our portfolio
                        w=1
                      
                monthly_profit= w*(spread[a]-spread[a-1])
                profits.append(monthly_profit)
                
                if weight_series[a-1]!=0:
                    monthly_return= w*(spread[a]/spread[a-1])-1
                    monthly_return_series.append(monthly_return)
               
               
                weight_series.append(w)
                    
                # We unwind our position whenever the spread reverts to its historical mean value
                if w>0:
                    if spread[a]>average:
                        w=0
                if w<0:
                    if spread[a]<average:
        
                        w=0
                
                    
        
        res[pair_i]=profits
        weights_mtx[pair_i]=weight_series
    
    return res, weights_mtx, monthly_return_series
    

In [16]:
#### We are now capable of producing and storing series of 

#### We can now observe the results of the trading strategy over a period of 6 months, after a training period of a year.

In [17]:
Profit_Matrix(data,group_1)[0]

IndexError: list index out of range

In [ ]:
Profit_Matrix(data,group_1)[0].describe()

Pairs,"(BRO, MCO)","(BRO, FITB)","(RJF, WFC)","(BEN, SPGI)","(BEN, FITB)"
count,125.000000,125.000000,125.000000,125.000000,125.000000
mean,-0.009253,0.007474,0.007316,-0.001309,-0.011662
std,0.133575,0.099594,0.238465,0.205919,0.160881
min,-0.416827,-0.310946,-0.598894,-0.747098,-0.665021
25%,-0.071308,-0.061213,-0.159509,-0.128649,-0.104259
50%,-0.002326,0.006967,-0.009575,0.025911,0.004868
75%,0.070670,0.075374,0.130767,0.131330,0.079429
max,0.347431,0.275914,0.739493,0.578046,0.328086


In [ ]:
Profit_Matrix(data,group_1)[1].describe()

Pairs,"(BRO, MCO)","(BRO, FITB)","(RJF, WFC)","(BEN, SPGI)","(BEN, FITB)"
count,125.0,125.00000,125.0,125.000000,125.0
mean,1.0,0.98400,1.0,-0.968000,-1.0
std,0.0,0.12598,0.0,0.217612,0.0
min,1.0,0.00000,1.0,-1.000000,-1.0
25%,1.0,1.00000,1.0,-1.000000,-1.0
50%,1.0,1.00000,1.0,-1.000000,-1.0
75%,1.0,1.00000,1.0,-1.000000,-1.0
max,1.0,1.00000,1.0,1.000000,-1.0


In [ ]:
def Total_Profits(df,pairs):
    
    pf_mtx=Profit_Matrix(df,pairs)[0]

    total_profits_list=[]
    for i in range((len(pairs))):
        total_profits_list.append(sum(pf_mtx[pairs[i]]))
    pf_mtx=pd.DataFrame(list(zip(pairs,total_profits_list)),columns=['Pairs','P&L'])
    total=sum(total_profits_list)
    return pf_mtx,total


In [ ]:
Group_1_Profits=Profit_Matrix(data,group_1)[0]
Total_Profits(data,group_1)

(         Pairs       P&L
 0   (BRO, MCO) -1.156569
 1  (BRO, FITB)  0.934216
 2   (RJF, WFC)  0.914490
 3  (BEN, SPGI) -0.163596
 4  (BEN, FITB) -1.457766,
 -0.929224783896335)

In [ ]:
Total_Profits(data,group_2)

(          Pairs       P&L
 0    (BRO, MCO) -1.156569
 1   (BRO, FITB)  0.934216
 2    (RJF, WFC)  0.914490
 3   (BEN, SPGI) -0.163596
 4   (BEN, FITB) -1.457766
 5    (BEN, MCO)  0.940475
 6    (BEN, BRO) -0.732071
 7   (HBAN, JPM) -0.324865
 8   (HBAN, WFC)  0.413990
 9   (HBAN, STT)  0.418100
 10  (HBAN, TFC) -0.280400
 11  (HBAN, KEY) -0.138562
 12  (HBAN, PNC)  0.024003
 13  (HBAN, HIG) -0.224666
 14  (HBAN, TRV) -0.136737
 15  (HBAN, IVZ)  0.273990
 16  (HBAN, PGR) -1.333420
 17   (AFL, TRV)  0.957420
 18    (AFL, BK)  0.804311
 19    (HBAN, L) -1.384202,
 -1.651858311506332)

In [ ]:
Total_Profits(data,group_3)

(          Pairs       P&L
 0     (AON, CB) -1.765773
 1    (AON, RJF) -2.469696
 2   (AON, HBAN) -1.318027
 3    (AON, BLK) -2.567074
 4    (AON, BRO)  2.164823
 5    (JPM, MMC)  1.461842
 6   (JPM, ZION)  0.884348
 7    (JPM, STT)  1.990969
 8    (JPM, KEY)  0.866244
 9    (JPM, LNC) -0.716085
 10   (JPM, TRV)  0.661259
 11   (FITB, RE)  0.352100
 12  (FITB, WRB) -2.514417
 13   (JPM, WRB)  1.457976
 14   (PNC, RJF) -0.306730
 15  (MMC, NTRS) -2.065623
 16  (MMC, TROW)  6.080253
 17   (MMC, STT) -1.523848
 18    (BK, COF)  2.048575
 19    (BK, LNC)  7.404230,
 10.125347626086656)

## Conclusions

#### Despite this Notebook has a great defect in being unable to provide the capacity to compute the monthly returns to actual capital employed, it still offers the capacity to compute the series of P&L values he would encounter by running the Pair Trading strategy referenced in this. With this further step, we would be able to compute Sharpe ratios and compare the application of this strategy over a large pannel of sectors and periods, in order to verify the hypotheses of Gatev, Goetzmann and Rouwenhorst on whether lower SSEs pairs overperformed the others, as well as the role of transaction costs in the decline of the profitability of such strategies, to the profit also of more complex quantitative trading strategies.

In [ ]:
print("Total profits for group 1 over 6 months is ", 4*Total_Profits(data,group_1)[1],'$')
print("Total profits for group 2 over 6 months is ", Total_Profits(data,group_2)[1],'$')
print("Total profits for group 3 over 6 months is ", Total_Profits(data,group_3)[1],'$')

Total profits for group 1 over 6 months is  -3.71689913558534 $
Total profits for group 2 over 6 months is  -1.651858311506332 $
Total profits for group 3 over 6 months is  10.125347626086656 $
